<a href="https://colab.research.google.com/github/annvorosh/GB/blob/ML_Med/%D1%83%D0%B4%D0%B0%D0%BB%D0%B8%D1%82%D1%8C_%22UNET_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в нейронные сети

## Урок 6. Сегментация


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, BatchNormalization, Concatenate, Conv2DTranspose, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from sklearn.model_selection import train_test_split
from PIL import Image

from tensorflow.keras import layers
import zipfile

In [52]:
# Константы
IMG_WIDTH = 336                       # Ширина изображения
IMG_HEIGHT = 336                      # Высота изображения
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT)   # Формат изображения (ширина, высота)
CLASSES = 2                           # Число классов сегментирования (полип и фон)
IMG_CHANNELS = 3
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

In [5]:
# Путь к файлу архива
zip_file_path = '/content/archive.zip'
extracted_dir_path = '/content/kvasir_capsule_seg'

# Распаковка архива
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

In [6]:
# Путь к папкам с изображениями и масками
extracted_dir_path = '/content/kvasir_capsule_seg'
images_dir = os.path.join(extracted_dir_path, 'Kvasir-Capsule/images')
masks_dir = os.path.join(extracted_dir_path, 'Kvasir-Capsule/masks')

# Получение списка файлов
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg')])
mask_files = sorted([f for f in os.listdir(masks_dir) if f.endswith('.jpg')])

# Создание датафрейма для удобства работы
data_df = pd.DataFrame({
    'image': [os.path.join(images_dir, f) for f in image_files],
    'mask': [os.path.join(masks_dir, f) for f in mask_files]
})



In [7]:
# Разделение на обучающую и тестовую выборки
train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=42)



In [9]:
# Загрузка данных
def load_data(df):
    images = []
    masks = []
    for _, row in df.iterrows():
        image = Image.open(row['image']).convert('RGB')  # Преобразование изображения в RGB
        mask = Image.open(row['mask']).convert('RGB')    # Преобразование маски в RGB
        image = tf.convert_to_tensor(np.array(image))    # Преобразование изображения в тензор
        mask = tf.convert_to_tensor(np.array(mask))      # Преобразование маски в тензор
        image = tf.image.resize(image, IMG_SHAPE)        # Изменение размера изображения
        mask = tf.image.resize(mask, IMG_SHAPE)          # Изменение размера маски
        images.append(image)
        masks.append(mask)
    return np.array(images), np.array(masks)

X_train, Y_train = load_data(train_df)
X_test, Y_test = load_data(test_df)



### Функции предобработки и пост обработки данных

In [39]:
def convert_from_color(arr_3d, palette):
    arr_2d = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.uint8)
    for c, i in palette.items():
        arr_2d[arr_3d[:, :, 0] == c[0]] = i
    return arr_2d


def preprocess_masks(masks, num_classes):
    processed_masks = []
    palette = {
        (0, 0, 0): 0,        # Фон (чёрный)
        (255, 255, 255): 1   # Полип (белый)
    }
    for mask in masks:
        mask = mask[:, :, :3]  # Убираем альфа-канал, если он есть
        mask = convert_from_color(mask, palette)  # Преобразование цвета в метки классов
        mask = tf.keras.utils.to_categorical(mask, num_classes)
        processed_masks.append(mask)
    return np.array(processed_masks)


In [40]:
# Подготовка данных для нейронной сети
X_train = X_train / 255.0  # Нормализация
X_test = X_test / 255.0    # Нормализация
Y_train = preprocess_masks(Y_train, CLASSES)
Y_test = preprocess_masks(Y_test, CLASSES)



### Предобработка исходных изображений и сегментированных изображений в ответ сети

### Объявление топологии нейронной сети, компиляция и обучение

In [58]:
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, Dropout, Concatenate, LeakyReLU
from tensorflow.keras.models import Model

def unet_model(image_size=(336, 336), output_classes=1):
    input_layer = Input(shape=image_size + (3,))

    # Encoder
    conv1 = Conv2D(64, 4, activation=LeakyReLU(), strides=2, padding='same', kernel_initializer='glorot_normal', use_bias=False)(input_layer)
    conv1_1 = Conv2D(128, 4, activation=LeakyReLU(), strides=2, padding='same', kernel_initializer='glorot_normal', use_bias=False)(conv1)
    conv2 = Conv2D(256, 4, activation=LeakyReLU(), strides=2, padding='same', kernel_initializer='glorot_normal', use_bias=False)(conv1_1)
    conv3 = Conv2D(512, 4, activation=LeakyReLU(), strides=2, padding='same', kernel_initializer='glorot_normal', use_bias=False)(conv2)
    conv4 = Conv2D(512, 4, activation=LeakyReLU(), strides=2, padding='same', kernel_initializer='glorot_normal', use_bias=False)(conv3)
    conv5 = Conv2D(512, 4, activation=LeakyReLU(), strides=2, padding='same', kernel_initializer='glorot_normal', use_bias=False)(conv4)
    conv6 = Conv2D(512, 4, activation=LeakyReLU(), strides=2, padding='same', kernel_initializer='glorot_normal', use_bias=False)(conv5)

    # Decoder
    up1 = Concatenate()([Conv2DTranspose(512, 4, activation='relu', strides=2, padding='same', kernel_initializer='glorot_normal', use_bias=False)(conv6), conv5])
    up1 = BatchNormalization()(up1)
    up1 = Dropout(0.25)(up1)

    up2 = Concatenate()([Conv2DTranspose(512, 4, activation='relu', strides=2, padding='same', kernel_initializer='glorot_normal', use_bias=False)(up1), conv4])
    up2 = BatchNormalization()(up2)
    up2 = Dropout(0.25)(up2)

    up3 = Concatenate()([Conv2DTranspose(512, 4, activation='relu', strides=2, padding='same', kernel_initializer='glorot_normal', use_bias=False)(up2), conv3])
    up3 = BatchNormalization()(up3)
    up3 = Dropout(0.25)(up3)

    up4 = Concatenate()([Conv2DTranspose(256, 4, activation='relu', strides=2, padding='same', kernel_initializer='glorot_normal', use_bias=False)(up3), conv2])
    up4 = BatchNormalization()(up4)

    up5 = Concatenate()([Conv2DTranspose(128, 4, activation='relu', strides=2, padding='same', kernel_initializer='glorot_normal', use_bias=False)(up4), conv1_1])
    up5 = BatchNormalization()(up5)

    up6 = Concatenate()([Conv2DTranspose(64, 4, activation='relu', strides=2, padding='same', kernel_initializer='glorot_normal', use_bias=False)(up5), conv1])
    up6 = BatchNormalization()(up6)

    # Output
    output_layer = Conv2DTranspose(output_classes, 4, activation='sigmoid', strides=2, padding='same', kernel_initializer='glorot_normal')(up6)

    model = Model(inputs=input_layer, outputs=output_layer)

    return model


In [59]:
model = unet_model()

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 12, 12, 512), (None, 11, 11, 512)]

In [ ]:
#Компилируем модель
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Обучение модели
history = model.fit(X_train, Y_train, batch_size=1, epochs=1, validation_data=(X_test, Y_test))



In [ ]:
# Визуализация точности обучения
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Точность модели')
plt.ylabel('Точность')
plt.xlabel('Количество эпох')
plt.legend(['Обучение', 'Тест'], loc='upper left')
plt.show()



### predict для двух тестовых картинок

In [ ]:
# Прогнозирование для тестовых изображений
out = model.predict(X_test[:2], batch_size=1)




In [ ]:
# Визуализация результатов сегментации для тестовых изображений
for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.imshow(convert_to_color(np.argmax(Y_test[i], axis=-1)))
plt.suptitle('Тестовая сегментация', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Визуализация предсказаний модели
for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.imshow

Точность модели 0.8628 при 100 эпохах обучения.  
Полученный результат сегментации очень похож на тестовый.